In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf

import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Import our input dataset
application_df = pd.read_csv('Resources/charity_data.csv')
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
application_df = application_df.drop(['EIN','NAME'], axis=1)

application_counts = application_df.APPLICATION_TYPE.value_counts()

replace_application = list(application_counts[application_counts < 528].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [3]:
classification_counts = application_df.CLASSIFICATION.value_counts()

replace_class = list(classification_counts[classification_counts < 1883].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [4]:
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [5]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### ATTEMPT1: Remove SPECIAL_CONSIDERATIONS_N

In [6]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL","SPECIAL_CONSIDERATIONS_N"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 80)                3440      
_________________________________________________________________
dense_27 (Dense)             (None, 30)                2430      
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 31        
Total params: 5,901
Trainable params: 5,901
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

In [29]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 961us/step - loss: 26120.2146 - accuracy: 0.5005
Epoch 2/100
152/804 [====>.........................] - ETA: 0s - loss: 20113.2472 - accuracy: 0.4975
Epoch 00002: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 15993.6354 - accuracy: 0.5020
Epoch 3/100
356/804 [============>.................] - ETA: 0s - loss: 12223.6919 - accuracy: 0.4987
Epoch 00003: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 27695.4921 - accuracy: 0.4924
Epoch 4/100
563/804 [====================>.........] - ETA: 0s - loss: 2553.4580 - accuracy: 0.5089
Epoch 00004: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 3133.2378 - accuracy: 0.5102
Epoch 5/100
773/804 [===========================>..] - ETA: 0s - loss: 0.6692 - accuracy: 0.5319
Epoch 00005: saving model to checkpoints\weig

804/804 [==============================] - 1s 1ms/step - loss: 0.6907 - accuracy: 0.5350
Epoch 39/100
435/804 [===============>..............] - ETA: 0s - loss: 0.6904 - accuracy: 0.5375
Epoch 00039: saving model to checkpoints\weights.39.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6908 - accuracy: 0.5345
Epoch 40/100
623/804 [======================>.......] - ETA: 0s - loss: 0.6918 - accuracy: 0.5271
Epoch 00040: saving model to checkpoints\weights.40.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6916 - accuracy: 0.5282
Epoch 41/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6914 - accuracy: 0.5301
Epoch 42/100
  1/804 [..............................] - ETA: 0s - loss: 0.7156 - accuracy: 0.3438
Epoch 00042: saving model to checkpoints\weights.42.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6915 - accuracy: 0.5288
Epoch 43/100
220/804 [=======>......................] - ETA: 0s - loss: 0.692

695/804 [========================>.....] - ETA: 0s - loss: 0.6911 - accuracy: 0.5325
Epoch 00076: saving model to checkpoints\weights.76.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 77/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6904 - accuracy: 0.5374
Epoch 78/100
 72/804 [=>............................] - ETA: 1s - loss: 0.6930 - accuracy: 0.5159
Epoch 00078: saving model to checkpoints\weights.78.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6915 - accuracy: 0.5288
Epoch 79/100
268/804 [=========>....................] - ETA: 1s - loss: 0.6910 - accuracy: 0.5327
Epoch 00079: saving model to checkpoints\weights.79.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 80/100
479/804 [================>.............] - ETA: 0s - loss: 0.6913 - accuracy: 0.5306 E
Epoch 00080: saving model to checkpoints\weights.80.hdf5
804/804 [=============

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6684 - accuracy: 0.6980
Loss: 0.6683541536331177, Accuracy: 0.6979591846466064


In [33]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimzation.h5")

### ATTEMPT2: Remove SPECIAL_CONSIDERATIONS_N, add neurons

In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  90
hidden_nodes_layer2 = 30

nn_two = tf.keras.models.Sequential()

# First hidden layer
nn_two.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
# Second hidden layer
nn_two.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_two.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_two.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 90)                3870      
_________________________________________________________________
dense_4 (Dense)              (None, 30)                2730      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 31        
Total params: 6,631
Trainable params: 6,631
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compile the model
nn_two.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model = nn_two.fit(X_train,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 950us/step - loss: 40377.5033 - accuracy: 0.4982
Epoch 2/100
804/804 [==============================] - 1s 976us/step - loss: 53233.0668 - accuracy: 0.5073
Epoch 3/100
804/804 [==============================] - 1s 981us/step - loss: 56570.4530 - accuracy: 0.5132
Epoch 4/100
804/804 [==============================] - 1s 983us/step - loss: 22009.8663 - accuracy: 0.5246
Epoch 5/100
804/804 [==============================] - 1s 979us/step - loss: 3640.5372 - accuracy: 0.5045
Epoch 6/100
804/804 [==============================] - 1s 973us/step - loss: 16972.8015 - accuracy: 0.5281
Epoch 7/100
804/804 [==============================] - 1s 978us/step - loss: 6157.9307 - accuracy: 0.5309
Epoch 8/100
804/804 [==============================] - 1s 980us/step - loss: 0.6912 - accuracy: 0.5313
Epoch 9/100
804/804 [==============================] - 1s 970us/step - loss: 0.6915 - accuracy: 0.5288
Epoch 10/100
804/804 [=========================

804/804 [==============================] - 1s 959us/step - loss: 0.6914 - accuracy: 0.5299
Epoch 79/100
804/804 [==============================] - 1s 963us/step - loss: 0.6908 - accuracy: 0.5345
Epoch 80/100
804/804 [==============================] - 1s 967us/step - loss: 0.6912 - accuracy: 0.5310
Epoch 81/100
804/804 [==============================] - 1s 962us/step - loss: 0.6905 - accuracy: 0.5369
Epoch 82/100
804/804 [==============================] - 1s 965us/step - loss: 0.6903 - accuracy: 0.5385
Epoch 83/100
804/804 [==============================] - 1s 955us/step - loss: 0.6912 - accuracy: 0.5318
Epoch 84/100
804/804 [==============================] - 1s 967us/step - loss: 0.6908 - accuracy: 0.5347
Epoch 85/100
804/804 [==============================] - 1s 960us/step - loss: 0.6908 - accuracy: 0.5344
Epoch 86/100
804/804 [==============================] - 1s 967us/step - loss: 0.6911 - accuracy: 0.5325
Epoch 87/100
804/804 [==============================] - 1s 961us/step - loss:

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_two.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7722 - accuracy: 0.5362
Loss: 0.7721960544586182, Accuracy: 0.5362099409103394


### ATTEMPT3: Remove SPECIAL_CONSIDERATIONS_N, add hidden layer

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn_three = tf.keras.models.Sequential()

# First hidden layer
nn_three.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
# Second hidden layer
nn_three.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_three.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn_three.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_three.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 80)                3440      
_________________________________________________________________
dense_7 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_8 (Dense)              (None, 10)                310       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 11        
Total params: 6,191
Trainable params: 6,191
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn_three.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model = nn_three.fit(X_train,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 959us/step - loss: 78554.1545 - accuracy: 0.4910
Epoch 2/100
804/804 [==============================] - 1s 971us/step - loss: 6475.7812 - accuracy: 0.4926
Epoch 3/100
804/804 [==============================] - 1s 950us/step - loss: 80731.3788 - accuracy: 0.5124
Epoch 4/100
804/804 [==============================] - 1s 988us/step - loss: 8103.4073 - accuracy: 0.5003
Epoch 5/100
804/804 [==============================] - 1s 954us/step - loss: 7009.8875 - accuracy: 0.5307
Epoch 6/100
804/804 [==============================] - 1s 946us/step - loss: 0.6960 - accuracy: 0.5368
Epoch 7/100
804/804 [==============================] - 1s 947us/step - loss: 0.6917 - accuracy: 0.5358
Epoch 8/100
804/804 [==============================] - 1s 952us/step - loss: 0.6912 - accuracy: 0.5326
Epoch 9/100
804/804 [==============================] - 1s 955us/step - loss: 0.6909 - accuracy: 0.5332
Epoch 10/100
804/804 [==============================] - 

804/804 [==============================] - 1s 961us/step - loss: 0.6916 - accuracy: 0.5284
Epoch 80/100
804/804 [==============================] - 1s 953us/step - loss: 0.6912 - accuracy: 0.5317
Epoch 81/100
804/804 [==============================] - 1s 931us/step - loss: 0.6911 - accuracy: 0.53190s - loss: 0.6912 - accura
Epoch 82/100
804/804 [==============================] - 1s 948us/step - loss: 0.6913 - accuracy: 0.5310
Epoch 83/100
804/804 [==============================] - 1s 948us/step - loss: 0.6908 - accuracy: 0.5342
Epoch 84/100
804/804 [==============================] - 1s 942us/step - loss: 0.6915 - accuracy: 0.5290
Epoch 85/100
804/804 [==============================] - 1s 949us/step - loss: 0.6910 - accuracy: 0.5330
Epoch 86/100
804/804 [==============================] - 1s 939us/step - loss: 0.6912 - accuracy: 0.5312
Epoch 87/100
804/804 [==============================] - 1s 946us/step - loss: 0.6916 - accuracy: 0.5282
Epoch 88/100
804/804 [=============================

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_three.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6813 - accuracy: 0.5840
Loss: 0.6813358068466187, Accuracy: 0.5840232968330383


### Attempt 4: Remove SPECIAL_CONSIDERATIONS, switch second layer to sigmoid

In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn_four = tf.keras.models.Sequential()

# First hidden layer
nn_four.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
# Second hidden layer
nn_four.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Output layer
nn_four.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_four.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 80)                3440      
_________________________________________________________________
dense_24 (Dense)             (None, 30)                2430      
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 31        
Total params: 5,901
Trainable params: 5,901
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the model
nn_four.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model = nn_four.fit(X_train,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 935us/step - loss: 0.6944 - accuracy: 0.5279
Epoch 2/100
804/804 [==============================] - 1s 976us/step - loss: 0.6911 - accuracy: 0.5342
Epoch 3/100
804/804 [==============================] - 1s 968us/step - loss: 0.6913 - accuracy: 0.5325
Epoch 4/100
804/804 [==============================] - 1s 972us/step - loss: 0.6915 - accuracy: 0.5312
Epoch 5/100
804/804 [==============================] - 1s 979us/step - loss: 0.6917 - accuracy: 0.5285
Epoch 6/100
804/804 [==============================] - 1s 987us/step - loss: 0.6913 - accuracy: 0.5327
Epoch 7/100
804/804 [==============================] - 1s 967us/step - loss: 0.6917 - accuracy: 0.5281
Epoch 8/100
804/804 [==============================] - 1s 968us/step - loss: 0.6914 - accuracy: 0.5318
Epoch 9/100
804/804 [==============================] - 1s 966us/step - loss: 0.6915 - accuracy: 0.5308
Epoch 10/100
804/804 [==============================] - 1s 974us/step - l

804/804 [==============================] - 1s 997us/step - loss: 0.6907 - accuracy: 0.5362
Epoch 80/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6914 - accuracy: 0.5331
Epoch 81/100
804/804 [==============================] - 1s 997us/step - loss: 0.6909 - accuracy: 0.5344
Epoch 82/100
804/804 [==============================] - 1s 1000us/step - loss: 0.6918 - accuracy: 0.5298
Epoch 83/100
804/804 [==============================] - 1s 953us/step - loss: 0.6912 - accuracy: 0.5331
Epoch 84/100
804/804 [==============================] - 1s 954us/step - loss: 0.6920 - accuracy: 0.5288
Epoch 85/100
804/804 [==============================] - 1s 995us/step - loss: 0.6912 - accuracy: 0.5336
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6907 - accuracy: 0.5364
Epoch 87/100
804/804 [==============================] - 1s 951us/step - loss: 0.6905 - accuracy: 0.5373
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.69

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_four.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7208 - accuracy: 0.4325
Loss: 0.7207649350166321, Accuracy: 0.43253645300865173
